# Anonymous functions and decorators
MCS 275 Spring 2024 - David Dumas

## Lambda   $\lambda$

### Named function

In [3]:
def f(x):
    "Compute the square of `x`"
    return x*x

* Create a function object that can take a value x and return x*x
* Add docstring info to that object
* Create a variable called f that refers to this new object

In [4]:
f(7)

49

In [5]:
f

<function __main__.f(x)>

In [6]:
type(f)

function

In [7]:
help(f)

Help on function f in module __main__:

f(x)
    Compute the square of `x`



### Unnamed (anonymous) function

In mathematics, a function can be described without giving it a name using the notation $x \mapsto x^2$, which is read aloud as "$x$ goes to $x^2$" or "$x$ maps to $x^2$.

Many programming languages allow a similar construction.  In Python, and in several other languages, this construction is named **lambda**, after the greek letter $\lambda$ which was used as a symbol for function definitions in a formal system (the *lambda calculus*) introduced by mathematician Alonzo Church in the 1930s.

In [9]:
# x maps to x**2
lambda x : x**2

<function __main__.<lambda>(x)>

This action returns a function object, which isn't very useful unless we do something with it.  We could assign to to a symbol;

In [10]:
g = lambda x : x**2

Now `g` is just like `f` we defined earlier.  It's a function.

In [11]:
g(8)

64

It's strictly worse, though.  It doesn't know its name and it has no docstring.

In [12]:
g.__name__

'<lambda>'

In [13]:
help(g)

Help on function <lambda> in module __main__:

<lambda> lambda x



### Aside

Markdown cells in Python notebooks support LaTeX, a document language that has rich mathematical typesetting capabilities.  Simply put inline mathematical expressions between `$` characters, e.g. `$\sqrt{x^5-1}$` produces $\sqrt{x^5-1}$.  Displayed mathematical expressions (that should be indented and separated from the surrounding text) go between `$` caracters, e.g.
```
$$
\int_0^\infty f(x)\, dx = 2 \pi
$$
```
produces
$$
\int_0^\infty f(x)\, dx = 2 \pi
$$

## When to use `lambda`

It's ideal for "quick" functions that are defined and used exactly once.  In that case, having to choose a name and put the definition on a different line can make it harder to understand the program.  In such cases, `lambda` gives a convenient shorthand.

Using custom orders in `list.sort`, `sorted`, `max`, and `min` is a typical example.  They accept a kwarg `key` that is a function that gets applied to elements before comparison.

In [15]:
L = [ "computer", "banana", "car", "avoidance", "zebra", "antidisestablishmentarianism"]

In [16]:
max(L) # last in dictionary order

'zebra'

In [17]:
min(L) # first in dictionary order

'antidisestablishmentarianism'

In [18]:
max(L,key=len) # the longest word

'antidisestablishmentarianism'

In [19]:
def num_distinct_chars(s):
    "Returns the number of distinct characters in s"
    return len(set(s))

In [20]:
min(L,key=num_distinct_chars) # One of the words with the fewest distinct characters

'banana'

In [21]:
# Same as above, but using lambda
min(L,key = lambda s:len(set(s)) )

'banana'

In [22]:
# The variable name doesn't matter
min(L,key = lambda t:len(set(t)) )

'banana'

In [23]:
# L, but sorted according to how far apart in the alphabet the most distant pair
# of letters are
sorted(L, key = lambda s : ord(max(s))-ord(min(s)))

['banana',
 'car',
 'computer',
 'antidisestablishmentarianism',
 'avoidance',
 'zebra']

Here, "banana" appears early in the sorted list because its most distant letters are `n` and `a`, which are only `ord("n")-ord("a")=13` steps apart.  The function `ord` converts a character to its unicode code point number.

`lambda` is also useful for partial application.  For example, suppose you have a function

In [26]:
def display_alert(title,bgcolor,message):
    """
    Open and highlight a centered window showing an urgent alert
    with given `title` and `message`.  The window's background
    color is `bgcolor`.
    """
    # code omitted
    

Then if you need a function that takes only one argument and displays a big red error message, you can use the expression

In [27]:
lambda msg: display_alert("ERROR","#FF5555",msg)

<function __main__.<lambda>(msg)>

## Decorators 🎀

### Functions as arguments

In [29]:
def dotwice(f):
    """Call zero-arg function `f` twice"""
    f()
    f()

In [30]:
def hello():
    print("Hello world.")
    # return None

In [31]:
hello()

Hello world.


In [32]:
hello()
hello()

Hello world.
Hello world.


In [33]:
dotwice(hello) # pass function object `hello` to function `dotwice`

Hello world.
Hello world.


In [36]:
dotwice( hello() ) # dotwice( None ) -->   None() None()

Hello world.


TypeError: 'NoneType' object is not callable

Use `*args` to take any number of additional positional args.

In [37]:
def h(x,y,*args):
    print("x is",x)
    print("y is",y)
    print("args is",args)

In [38]:
h(1,2)

x is 1
y is 2
args is ()


In [39]:
h(1,2,4,8,"Hello",True,None)

x is 1
y is 2
args is (4, 8, 'Hello', True, None)


`**kwargs` is similar for keyword arguments.  Using these we can upgrade `dotwice` to work with functions that take arguments.

In [34]:
def dotwice(f,*args,**kwargs):
    """
    Call function `f` twice, giving the same
    set of arguments and kwargs each time.
    """
    f(*args,**kwargs)
    f(*args,**kwargs)

In [35]:
dotwice(print,"hello this is mcs",275,end="")

hello this is mcs 275hello this is mcs 275

In [36]:
print("hello this is mcs",275,end="")
print("hello this is mcs",275,end="")

hello this is mcs 275hello this is mcs 275

### Function factory

In [48]:
def power_function(n):   # integer |----->  function
    def david(x): # function inside a function!
        """Raise x to a power"""
        return x**n
    return david

In [49]:
square = power_function(2)
cube = power_function(3)

In [50]:
square

<function __main__.power_function.<locals>.inner(x)>

In [51]:
cube

<function __main__.power_function.<locals>.inner(x)>

In [52]:
square(5)  # 5**2

25

In [53]:
cube(7)  # 7**3

343

### Function factory that takes a function argument

In [37]:
def return_twice_doer(f):
    """Return a new function which calls f twice"""
    def inner(*args,**kwargs):
        """Call a certain function twice"""
        f(*args,**kwargs)
        f(*args,**kwargs)
    return inner

In [38]:
def concern():
    print("I'm a little bit worried")
    
def panic():
    print("OMG EVERYTHING IS TERRIBLE")

In [39]:
dotwice(concern)

I'm a little bit worried
I'm a little bit worried


In [40]:
dotwice(panic)

OMG EVERYTHING IS TERRIBLE
OMG EVERYTHING IS TERRIBLE


In [41]:
doubleconcern = return_twice_doer(concern)

In [42]:
doubleconcern()

I'm a little bit worried
I'm a little bit worried


In [43]:
doublepanic = return_twice_doer(panic)

In [44]:
doublepanic

<function __main__.return_twice_doer.<locals>.inner(*args, **kwargs)>

In [45]:
doublepanic()

OMG EVERYTHING IS TERRIBLE
OMG EVERYTHING IS TERRIBLE


Here's another example.  It take a function `f` and "wraps" it with `print` statements.

In [49]:
def wrap_with_notification(f):
    def inner(*args,**kwargs):
        print("I am about to call",f.__name__)
        res = f(*args,**kwargs)
        print("The call to",f.__name__,"finished")
        return res
    return inner

In [50]:
import time

def long_running_calculation(n):
    for i in range(n):
        time.sleep(1)
        print(i)
    return 275

In [51]:
long_running_calculation(4)

0
1
2
3


275

In [52]:
long_running_with_notification = wrap_with_notification(long_running_calculation)

In [53]:
long_running_with_notification(4)

I am about to call long_running_calculation
0
1
2
3
The call to long_running_calculation finished


275

But why choose a new name?  We could replace the original `long_running_calculation` with the wrapped one, overwriting it.

In [ ]:
long_running_calculation = wrap_with_notification(long_running_calculation)

Python has a dedicated syntax for doing both: Define a function, then replace it with the return value of some other function.

In [54]:
@wrap_with_notification
def number_of_digits(n):
    return len(str(n))

In [55]:
number_of_digits(481275)

I am about to call number_of_digits
The call to number_of_digits finished


6

This is roughly how the `@app.route` decorator works.  (This is not a real, working example.  It is just an outline suggesting how a decorator converts a function returning a string into a mechanism to handle HTTP requests.

In [75]:
def web_serverize(f):
    def inner():
        request_args = wait_for_relevant_HTTP_get()
        s = f(*request_args)
        send_string_as_HTTP_response(s)
    return inner

In [ ]:
@web_serverize
def home():
    return "<html><body>Hello!</body></html>"